In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
5%cd /content/drive/MyDrive/.Projektseminar Computerlinguistik/6Emoji Probing

/content/drive/MyDrive/.Projektseminar Computerlinguistik/6Emoji Probing


In [ ]:
%cd semeval2parquet

/content/drive/MyDrive/.Projektseminar Computerlinguistik/6Emoji Probing/semeval2parquet


In [ ]:
!ls

read_data.ipynb  us_test.labels  us_test.text  us_trial.labels	us_trial.text


# Fine Tuning Transformer for MultiClass Text Classification

In [ ]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = torch.device('cuda:0') if cuda.is_available() else 'cpu'
#device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
!ls

read_data.ipynb  us_test.labels  us_test.text  us_trial.labels	us_trial.text


In [ ]:
!head -10 us_test.text

en Pelham Parkway
The calm before...... | w/ sofarsounds @user | : B. Hall.......#sofarsounds…
Just witnessed the great solar eclipse @ Tampa, Florida
This little lady is 26 weeks pregnant today! Excited for baby Cam to come! @ Springfield,…
Great road trip views! @ Shartlesville, Pennsylvania
CHRISTMAS DEALS BUY ANY 3 SMALL POMADES 1.5 OR 1.7 OZ RECEIVE THE F&amp;S COLLECTOR TIN &amp; COMB…
the #sisterstunt was mad real last night #MiaStaxxx #AndreaStaxxx #denverqueen #staxxxlife…
I'm starting to love shooting in the dark #brandonwolfel @ New York, New York
Let the sun shine through ️ 5x5 Feet #oilpainting #oiloncanvas #acrylicpainting #acryliconcanvas…
Still bitch im trill never been no fiend :@vibesbygallo #mustard @ Connecticut


In [ ]:
!head -10 us_test.labels

2
10
6
1
16
17
4
10
12
18


In [ ]:
text_path = "us_test.text"
text_list = []
with open(text_path, "r") as text_file:
    for line in text_file:
      line = line.strip()
      text_list.append(line)

print(len(text_list))



50000


In [ ]:
label_path = "us_test.labels"
label_list = []
with open(label_path, "r") as label_file:
    for line in label_file:
      line = line.strip()
      line = int(line)
      label_list.append(line)

print(len(label_list))
print(label_list[:2])

50000
[2, 10]


In [ ]:
print(set(label_list))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [ ]:
d = {'TITLE': text_list, 'CATEGORY': label_list, 'ENCODE_CAT' :label_list}
df = pd.DataFrame(data=d)
df

,TITLE,CATEGORY,ENCODE_CAT
0,en Pelham Parkway,2,2
1,The calm before...... | w/ sofarsounds @user |...,10,10
2,Just witnessed the great solar eclipse @ Tampa...,6,6
3,This little lady is 26 weeks pregnant today! E...,1,1
4,"Great road trip views! @ Shartlesville, Pennsy...",16,16
...,...,...,...
49995,@user @user @user #la #westhollywood #dtboy #l...,5,5
49996,Climbing subway stairs. That was nothing. #sta...,19,19
49997,Pops with Ms Drina at The Swanees Anniversary ...,6,6
49998,"We love ️ Soren! July 26, 2017 was her first d...",0,0


# Preparing the Dataset and Dataloader

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 250
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 12
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
num_classes = 20

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        title = str(self.data.TITLE[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (50000, 3)
TRAIN Dataset: (40000, 3)
TEST Dataset: (10000, 3)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# Creating the Neural Network for Fine Tuning





In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistillBERTClass(torch.nn.Module):
    def __init__(self, num_classes):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = DistillBERTClass(num_classes)
model.to(device)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Fine Tuning the *Model*

In [ ]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

import sys, os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

def train(epoch):

    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        #ids  = data['ids']
        #print(len(data['ids']))
        #sys.exit()
        #print(data['ids'])
        #print(data['targets'].shape)

        # ids = torch.randn((4, 512)).long().to(device)
        # mask = torch.randn((4, 512)).long().to(device)
        # targets = torch.arange(20)[:4].long().to(device)

        # print(ids.shape)


        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 3.0622735023498535
Training Accuracy per 5000 steps: 0.0
Training Loss per 5000 steps: 2.2935111876834133
Training Accuracy per 5000 steps: 30.508898220355928
The Total Accuracy for Epoch 0: 33.09
Training Loss Epoch: 2.213548289938271
Training Accuracy Epoch: 33.09
Training Loss per 5000 steps: 3.0819270610809326
Training Accuracy per 5000 steps: 0.0
Training Loss per 5000 steps: 2.0406952036115986
Training Accuracy per 5000 steps: 38.72725454909018
The Total Accuracy for Epoch 1: 39.5175
Training Loss Epoch: 2.0096871711343525
Training Accuracy Epoch: 39.5175
Training Loss per 5000 steps: 1.7023143768310547
Training Accuracy per 5000 steps: 50.0
Training Loss per 5000 steps: 1.8657194735224225
Training Accuracy per 5000 steps: 43.76124775044991
The Total Accuracy for Epoch 2: 43.9425
Training Loss Epoch: 1.867849233519286
Training Accuracy Epoch: 43.9425
Training Loss per 5000 steps: 1.7346200942993164
Training Accuracy per 5000 steps: 50.0
Training Loss

# Validating the Model

In [30]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    n_wrong = 0
    total = 0
    tr_loss = 0  # Initialize tr_loss
    nb_tr_steps = 0
    nb_tr_examples = 0

    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()  # Update tr_loss with the current loss
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)  # Fix the typo

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")

    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu


print('This is the validation section to print the accuracy and see how it performs')
print(
    'Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of '
    'pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss per 100 steps: 1.010585904121399
Validation Accuracy per 100 steps: 50.0
Validation Loss Epoch: 3.6396572663607447
Validation Accuracy Epoch: 38.51
Accuracy on test data = 38.51%


# Saving the Trained Model Artifacts for inference